# 1. Partititioning The Data
Instructions
Partition the dataset into 5 folds and store each row's fold in a new integer column named fold:
Fold 1 : rows from index 0 to 128, including both of those rows.
Fold 2 : rows from index 129 to 257, including both of those rows.
Fold 3 : rows from index 258 to 386, including both of those rows.
Fold 4 : rows from index 387 to 514, including both of those rows.
Fold 5 : rows from index 515 to 644, including both of those rows.
Display the first 5 rows and the last 5 rows of the Dataframe to confirm.
Hint
Use df.ix[index_slice, col_name] to mass assign a specific value for col_name for all of the rows in the index_slice. You can use this to set a value for a new column in the Dataframe as well.

In [1]:
import pandas as pd 
import numpy as np

admissions = pd.read_csv("admissions.csv")
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)

shuffled_index = np.random.permutation(admissions.index)
shuffled_admissions = admissions = admissions.loc[shuffled_index]
admissions = shuffled_admissions.reset_index()
admissions.head()

,index,gpa,gre,actual_label
0,203,3.419984,497.657334,0
1,324,3.120419,655.062350,0
2,27,3.122304,714.245338,0
3,393,2.489852,585.515283,0
4,24,2.848408,390.363563,0


In [2]:
admissions.ix[0:128, "fold"] = 1
admissions.ix[129:257, "fold"] = 2
admissions.ix[258:386, "fold"] = 3
admissions.ix[287:514, "fold"] = 4
admissions.ix[515:644, "fold"] = 5
admissions["fold"] = admissions["fold"].astype('int')

In [3]:
print(admissions.head())

   index       gpa         gre  actual_label  fold
0    203  3.419984  497.657334             0     1
1    324  3.120419  655.062350             0     1
2     27  3.122304  714.245338             0     1
3    393  2.489852  585.515283             0     1
4     24  2.848408  390.363563             0     1


In [4]:
print(admissions.tail())

     index       gpa         gre  actual_label  fold
639    250  3.493100  540.809834             0     5
640    121  2.799225  654.216375             0     5
641    272  3.495587  658.965386             0     5
642    463  3.064387  659.915493             1     5
643    462  3.625503  606.370700             1     5


# 2. First Iteration
Instructions Train a logistic regression model using the gpa column as the sole feature from folds 2 to 5 as the training set.c
Use the model to make predictions on the test set and assign the predicted labels to labels.
Calculate the accuracy by comparing the predicted labels with the actual labels from the actual_label column on the test set.
Assign the accuracy value to iteration_one_accuracy.
Hint
Documentation for LogisticRegression.fit(). Mission on Logistic Regression.

In [5]:
from sklearn.linear_model import LogisticRegression

#training 
model = LogisticRegression()
train_iteration_one = admissions[admissions["fold"] != 1]
test_iteration_one = admissions[admissions["fold"] == 1]
model.fit(train_iteration_one[["gpa"]], train_iteration_one["actual_label"])

#predicting
labels = model.predict(test_iteration_one[["gpa"]])
test_iteration_one["predicted_label"] = labels

matches = test_iteration_one["predicted_label"] == test_iteration_one["actual_label"]
correct_predictions = test_iteration_one[matches]
iteration_one_accuracy = len(correct_predictions) / len(test_iteration_one)
print(iteration_one_accuracy)

0.6201550387596899


//anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 3. Function For Training Models
Instructions Write a function named train_and_test that takes in a Dataframe and a list of fold id values (1 to 5 in our case) and returns a list of accuracy values, e.g.:
[0.5, 0.5, 0.5, 0.5, 0.5]
Use the train_and_test function to return the list of accuracy values for the admissions Dataframe and assign to accuracies. e.g.:
accuracies = train_and_test(admissions, [1,2,3,4,5])
Compute the average accuracy and assign to average_accuracy.
average_accuracy should be a float value while accuracies should be a list of float values (one float value per iteration).
Use the variable inspector or the print function to display the values for accuracies and average_accuracy.
Hint
Within the function, use the fold id values to partition the Dataframe by the fold column

In [6]:
import numpy as np
fold_ids = [1,2,3,4,5]

def train_and_test(df, folds):
    fold_accuracies = []
    for fold in folds:
        model = LogisticRegression()
        train = df[df["fold"] != fold]
        test = df[df["fold"] == fold]
        model.fit(train[["gpa"]], train["actual_label"])
        labels = model.predict(test[["gpa"]])
        test["predicted_label"] = labels
        
        matches = test["predicted_label"] == test["actual_label"]
        correct_predictions = test[matches]
        fold_accuracies.append(len(correct_predictions) / len(test))
    return(fold_accuracies)

accuracies = train_and_test(admissions, fold_ids)
print(accuracies)
average_accuracy = np.mean(accuracies)
print(average_accuracy)

[0.6201550387596899, 0.6124031007751938, 0.6206896551724138, 0.6666666666666666, 0.6511627906976745]
0.634215450414


//anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 4. Sklearn
Instructions
Create a new instance of the KFold class with the following properties:
n set to length of admissions,
5 folds,
shuffle set to True,
random seed set to 8 (so we can answer check using the same seed),
assigned to the variable kf.
Create a new instance of the LogisticRegression class and assign to lr.
Use the cross_val_score function to perform k-fold cross-validation:
using the LogisticRegression instance lr,
using the gpa column for training,
using the actual_label column as the target column,
returning an array of accuracy values (one value for each fold).
Assign the resulting array of accuracy values to accuracies, compute the average accuracy, and assign the average to average_accuracy.
Use the variable inspector or the print function to display the values for accuracies and average_accuracy.
Hint
Documentation on KFold. List of accepted scoring criteria for cross_val_score.

In [28]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

admissions = pd.read_csv("admissions.csv")
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)

kf = KFold(len(admissions), 5, shuffle=True, random_state=8)

lr = LogisticRegression()

accuracies = cross_val_score(lr, admissions[["gpa"]], admissions["actual_label"], scoring="accuracy", cv=kf)
average_accuracy = sum(accuracies) / len(accuracies)
print(accuracies)
print(average_accuracy)

[ 0.6124031   0.65891473  0.64341085  0.6744186   0.6328125 ]
0.644391957364
